#  following code is sample implementation of RNN using tf for Text


In [5]:
import numpy as np
import tensorflow as tf 
import time
import os

from six.moves import cPickle


from newModel import  Model
import codecs
import collections

import argparse

In [6]:
dataDir = 'data/alice' ## defining the directory of the dataset
inputEncoding = None ## https://docs.python.org/3/library/codecs.html#standard-encodings

logDir = "./logDir"
saveDir = "./dataSave"
rnnSize = 256   # size of the rnn 
noLayers = 2   # no of layers in RNN
model = "lstm" # lstm model
batchSize= 20  # size of batch
seqLength= 25  # length of the sequence
noEpoch =25 # epoch to save
saveEvery = 1000 # frequency of saving
gradClip =  .5
learningRate = 0.002
decayRate = 0.95
initFrom = None
gpuMem = 0.66

In [7]:
## loding the data into the memry
inputFile = os.path.join(dataDir,'input.txt')
vocabFile = os.path.join(dataDir,'vocab.pkl')
tensorFile =os.path.join(dataDir,'tensor.pkl')

In [8]:
with codecs.open(inputFile,"r",encoding=None) as f:
    data = f.read() ## this function will read the full sentences 

In [9]:
## now we are going to split the data based on spaces. NOTE:- there is no split which is happening based on 
##. or some other punctuation

xText = data.split()
print(1)

1


In [10]:
wordCount = collections.Counter(xText)
wordName = [x[0] for x in  wordCount.most_common()]
wordName = list(sorted(wordName))

vocab = {x:i for  i,x in enumerate(wordName)}
words = [x[0] for x in  wordCount.most_common()]
print("vocab List :", len(words))

with open(vocabFile, 'wb') as f:
    cPickle.dump((words), f)

vocab List : 6019


In [11]:
tensor = np.array(list(map(vocab.get, xText)))

# Save the data to data.npy
np.save(tensorFile, tensor)

print('tensor is:' + str(tensor))
print("It's shape: " + str(np.shape(tensor)))

tensor is:[1066  786  539 ... 1330 3953 2481]
It's shape: (29465,)


In [12]:
vocabSize = len(words)

In [13]:
## creating the no of batches tp be processed
numBatch = int(tensor.size/(seqLength*batchSize))

In [14]:
## data to be processed
tensor = tensor[:numBatch*batchSize*seqLength]
print (tensor.size)

29000


In [15]:
xData = tensor
yData = np.copy(tensor)



In [16]:
yData[:-1] = xData[1:] 
yData[-1] = xData[0]

In [17]:
print(xData)
print(yData)

[1066  786  539 ... 3405 5729 1474]
[ 786  539  530 ... 5729 1474 1066]


In [18]:
# print (xData.shape)
newData = xData.reshape(batchSize,-1)
newLabel = yData.reshape(batchSize, -1)
xBatchTrain = np.split(newData,numBatch,1)
yBatchTrain = np.split(newLabel,numBatch,1)

print(xBatchTrain[0].shape)
print(yBatchTrain[0].shape)


(20, 25)
(20, 25)


In [19]:
pointer =0

In [20]:
pointer = 0 ## batch pointer
with open(os.path.join(dataDir,"word_vocab.pkl"),"wb") as f:
    cPickle.dump((words,vocab),f)

In [21]:
tf.reset_default_graph()
model = Model(dataDir=dataDir,inputEncoding=inputEncoding,logDir=logDir,saveDir=saveDir,rnnSize=rnnSize,numLayers=noLayers,model=model,batchSize=batchSize,seqLength=seqLength,noEpoch=noEpoch,
				saveAfterEvery=saveEvery,gradClip=gradClip,learningRate=learningRate,decayRate=decayRate,gpuMemory=gpuMem,initFrom=initFrom,vocabSize=vocabSize)

In [22]:
merged = tf.summary.merge_all()
trainWriter = tf.summary.FileWriter(logDir)

In [23]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpuMem)


In [24]:
def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.int64)
  print (arg.shape)
  return arg

# Training the model in session environment 

In [30]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    ## add the graph in log dir
    trainWriter.add_graph(sess.graph)
    
    ## initialize all the global variable
    tf.global_variables_initializer().run()
    
    ## making a saver file to save all the variable
    saver = tf.train.Saver(tf.global_variables())
    
    
    ## creating a for loop itereating the epoch 
    
    for e in range(model.epochPointer.eval(),noEpoch):
        
        ## initializing the lr value of the model
        sess.run(tf.assign(model.lr,learningRate*(decayRate)**e))
        
        ## definig the initial state of the system
        state = sess.run(model.initialState)
        
        ## pointer for the batch
        pointer = 0
        
        ## pointer over the speed 
        speed =0 
        if initFrom is None:
            ## initFrom function is none
            assignOp = model.epochPointer.assign(e)
            sess.run(assignOp)
        else:
            assignOp = model.batchPointer.eval()
            initFrom = None
    
    
        ## for each epoch, for loop to interate over each batch (b)
        for b in range(pointer,numBatch):
            start = time.time()
            x, y = xBatchTrain[pointer],yBatchTrain[pointer]
            pointer+=1
            
            feed = {model.inputData:x,model.targets:y,model.initialState : state,model.batchTime : speed}
            print 
            summary,trainLoss,state,_,_ = sess.run([merged,model.cost,model.finalState,model.trainOp,model.incBatchPointerOp],feed)
            
            ## adding the summary writer  with the globalstep
            trainWriter.add_summary(summary,8*numBatch+b)
            
            speed = time.time()-start
            if (e * numBatch + b) % batchSize == 0:
                print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                    .format(e * numBatch + b,
                            noEpoch * numBatch,
                            e, trainLoss, speed))

                #save model:
                if (e * numBatch + b) % saveEvery == 0 \
                        or (e==noEpoch-1 and b == numBatch-1): # save for the last result
                    #define the path to the model
                    checkpoint_path = os.path.join(saveDir, 'model_test.ckpt')
                    #save the model, woth increment ()
                    saver.save(sess, checkpoint_path, global_step = e * numBatch + b)
                    print("model saved to {}".format(checkpoint_path))
        
        #close the session
    trainWriter.close()
            
            
    

0/1450 (epoch 0), train_loss = 8.702, time/batch = 0.425
model saved to ./dataSave\model_test.ckpt
20/1450 (epoch 0), train_loss = 7.846, time/batch = 0.111
40/1450 (epoch 0), train_loss = 7.269, time/batch = 0.106
60/1450 (epoch 1), train_loss = 7.314, time/batch = 0.104
80/1450 (epoch 1), train_loss = 6.969, time/batch = 0.105
100/1450 (epoch 1), train_loss = 6.853, time/batch = 0.119
120/1450 (epoch 2), train_loss = 7.114, time/batch = 0.110
140/1450 (epoch 2), train_loss = 6.986, time/batch = 0.109
160/1450 (epoch 2), train_loss = 6.836, time/batch = 0.105
180/1450 (epoch 3), train_loss = 6.934, time/batch = 0.106
200/1450 (epoch 3), train_loss = 6.952, time/batch = 0.112
220/1450 (epoch 3), train_loss = 6.948, time/batch = 0.112
240/1450 (epoch 4), train_loss = 6.852, time/batch = 0.116
260/1450 (epoch 4), train_loss = 7.029, time/batch = 0.104
280/1450 (epoch 4), train_loss = 6.810, time/batch = 0.101
300/1450 (epoch 5), train_loss = 7.216, time/batch = 0.099
320/1450 (epoch 5), 

#  code for testing the system using basic words - second phase (run separately)

In [2]:
import numpy as np
import tensorflow as tf 
import time 
import os
from six.moves import cPickle

from newModel import  Model

In [12]:
saveDir = "./dataSave" ## directory to load data from 

n =200 ## no of words to sample from

prime = "Alice"

sample = 3 #0 to use max at each timestep, 1 to sample at each timestep, 2 to sample on spaces


In [13]:
inputEncoding = None ## https://docs.python.org/3/library/codecs.html#standard-encodings

dataDir = 'data/alice' ## defining the directory of the dataset
logDir = "./logDir"
saveDir = "./dataSave"
rnnSize = 256   # size of the rnn 
noLayers = 2   # no of layers in RNN
model = "lstm" # lstm model
batchSize= 20  # size of batch
seqLength= 25  # length of the sequence
noEpoch =25 # epoch to save
saveEvery = 1000 # frequency of saving
gradClip =  .5
learningRate = 0.002
decayRate = 0.95
initFrom = None
gpuMem = 0.66

In [14]:
with open(os.path.join(dataDir, 'word_vocab.pkl'), 'rb') as f:
        words, vocab = cPickle.load(f)

In [15]:
vocabSize = len(words)

In [16]:
tf.reset_default_graph()
model = Model(dataDir=dataDir,inputEncoding=inputEncoding,logDir=logDir,saveDir=saveDir,rnnSize=rnnSize,numLayers=noLayers,model=model,batchSize=batchSize,seqLength=seqLength,noEpoch=noEpoch,
				saveAfterEvery=saveEvery,gradClip=gradClip,learningRate=learningRate,decayRate=decayRate,gpuMemory=gpuMem,initFrom=initFrom,vocabSize=vocabSize,infer=True)

In [17]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    #then we define the Saver to retrieve the model
    saver = tf.train.Saver(tf.global_variables())
    
    #we retrieve the checkpoint of the stored model:
    ckpt = tf.train.get_checkpoint_state(saveDir)
    if ckpt and ckpt.model_checkpoint_path:
        #we restore the model
        saver.restore(sess, ckpt.model_checkpoint_path)

        #we create the results
        results = model.sample(sess, words, vocab, n, prime, sample)

INFO:tensorflow:Restoring parameters from ./dataSave\model_test.ckpt-1000


In [18]:
print(results)

Alice dodged distance--but clever newspapers, strength, clever clever him--How whiting!' leaves please, encouraged started continued Beautiful, else's gravely. Time, larger: beautifully creation clever clever About paragraphs person silence, Stretching, them, altogether jogged Rabbit. Rabbit. encouraged NO quiet dream. This beauti--FUL Always to, brain; clever surprise encouraged Footman, beautifully beautifully (801) Northumbria--"' else's business!' ear. shall!' ring, '--it get" writing hadn't particularly held eagerly right ink, described appearing Hart, ink, choke North soldier eagerly creep dull them--and provisions. whiting!' Derision.' reach Him, whiting!' remarks,' thousand experiment beautifully TRADEMARK any) officer shower brought clever compliance. dream. because hot-tempered,' then--always doubtful body clever whiting!' this), FROM DISTRIBUTE previous me! might clever dream. Rabbit's--'Pat! way. frighten course--"I 90 Any with; head!"' eagerly low. furiously, clever glass;